### Библиотеки / данные

импортируем numpy и pandas

In [ ]:
import numpy as np
import pandas as pd

задаем некоторые настройки pandas, регулирующие формат вывода

In [ ]:
pd.options.display.max_rows = 10

- считываем данные 
- используем столбец Symbol в качестве индекса 
- считываем только те столбцы ['Symbol', 'Sector', 'Price', 'Book Value']

| Column Name        | Description
| ------------- |:-------------:|
|Symbol|Сокращенное название организации|
|Name|Полное название организации|
|Sector|Сектор экономики|
|Price|Стоимость акции|
|Dividend Yield|Дивидендная доходность|
|Price/Earnings|Цена / прибыль|
|Earnings/Share|Прибыль на акцию|
|Book Value|Балансовая стоимость компании|
|52 week low|52-недельный минимум|
|52 week high|52-недельный максимум|
|Market Cap|Рыночная капитализация|
|EBITDA|**E**arnings **b**efore **i**nterest, **t**axes, **d**epreciation and **a**mortization|
|Price/Sales|Цена / объём продаж|
|Price/Book|Цена / балансовая стоимость|
|SEC Filings|Ссылка *sec.gov*|

In [ ]:
sp500 = pd.read_csv("data/sp500.csv",
                    index_col='Symbol',
                    usecols=['Symbol', 'Sector', 'Price', 'Book Value'])

In [ ]:
sp500.head()

считываем исторические данные о котировках акций

In [ ]:
omh = pd.read_csv('data/omh.csv', 
                  parse_dates=['Date'])

omh.set_index('Date', 
              inplace=True)

In [ ]:
omh.head()

### Сводка статистик

получаем сводку статистик для датафрейма, с которой работаем как с обычным датафреймом

In [ ]:
sp500.describe()

вычисляем сводку статистик для отдельного столбца Price

In [ ]:
sp500.Price.describe()

получаем сводку статистик для нечисловых данных

In [ ]:
sp500.Sector.describe()

метод info:

In [ ]:
sp500.info()

получаем сводную статистику для нечисловых данных

In [ ]:
sp500.Sector.value_counts(normalize=True)

### Арифметические операции

- задаем стартовое значение генератора случайных чисел для получения воспроизводимых результатов
- создаем объект DataFrame

In [ ]:
np.random.seed(123)
df = pd.DataFrame(np.random.randn(5, 4), 
                  columns=['A', 'B', 'C', 'D'])
df

умножаем все на 2, берём только абсолютные значения

In [ ]:
abs(df * 2)

вычитаем первую строку из каждой строки объекта DataFrame

In [ ]:
df

In [ ]:
df.iloc[0]

In [ ]:
df - df.iloc[0]

вычитаем объект DataFrame из объекта Series

In [ ]:
df.iloc[0] - df

- возьмём второе и третье поле 1-ой строки:
- добавляем столбец E
- смотрим, как применяется выравнивание в этой математической операции

In [ ]:
df

In [ ]:
s = df.iloc[0][1:3]
s['E'] = 0
s

In [ ]:
df + s

извлекаем строки в позициях с 1-й по 3-ю и только столбцы B и C <br>
по сути - извлекаем небольшой квадрат из середины df

In [ ]:
subframe = df[1:4][['B', 'C']].copy()
subframe

демонстрируем, как происходит выравнивание при выполнении операции вычитания

In [ ]:
df - subframe

извлекаем столбец A и вычитаем его из нашего датафрейма

In [ ]:
df.sub(df['A'], axis=0)

### Одномерные статистики

#### минимум / максимум

определяем максимальную цену для обеих акций

In [ ]:
omh[['MSFT', 'AAPL']].min()

определяем индекс, которому соответствует максимальная цена для обеих акций

In [ ]:
omh[['MSFT', 'AAPL']].idxmin()

#### cреднее значение / медиана / мода

<img src='..\images\moda-mediana.jpg'/>

вычисляем среднее значение для всех столбцов в датафрейме omh

In [ ]:
omh.mean()

вычисляем значение, усредненное по всем столбцам, для каждой строки (выведем первые 5)

In [ ]:
omh.mean(axis=1).head() 

вычисляем медиану значений для каждого столбца

In [ ]:
omh.median()

вычисляем моду для столбца Sector

In [ ]:
sp500.Sector.mode()

мод может быть несколько, поэтому результат операции - Series 

In [ ]:
s = pd.Series([1, 2, 3, 3, 5, 1])
s.mode()

#### [дисперсия](https://ru.wikipedia.org/wiki/Дисперсия_случайной_величины) / среднеквадратичное отклонение

вычисляем дисперсию значений в каждом столбце

In [ ]:
omh.var()

In [ ]:
(omh.MSFT**2 - omh.MSFT.mean()**2).sum() / (omh.shape[0]-1)

вычисляем среднеквадратичное отклонение

In [ ]:
omh.std()

In [ ]:
omh.MSFT.var()**0.5

#### [ковариация](https://ru.wikipedia.org/wiki/Ковариация) / [корреляция](https://ru.wikipedia.org/wiki/Корреляция)

вычисляем ковариацию между MSFT и AAPL

In [ ]:
omh.MSFT.cov(omh.AAPL)

вычисляем корреляцию между MSFT и AAPL

In [ ]:
omh.MSFT.corr(omh.AAPL)

In [ ]:
omh.MSFT.cov(omh.AAPL) / (omh.MSFT.std() * omh.AAPL.std())

либо можем получать матрицу ковариаций

In [ ]:
omh.corr()

### Преобразование данных 

#### дискретизация и квантилизация

генерируем 10000 случайных чисел из стандартного нормального распределения

In [ ]:
np.random.seed(123456)
dist = np.random.normal(size = 10000)
dist

выводим среднее и стандартное отклонение

In [ ]:
(dist.mean(), dist.std())

разбиваем на пять одинаковых по размеру групп (по размеру интервалов - не количеству наблюдений в группе!)

In [ ]:
bins = pd.cut(dist, 5)
bins

найдём длины соответствующих интервалов

In [ ]:
bins.categories

In [ ]:
[q.right - q.left for q in bins.categories]

генерируем 50 значений возраста в диапазоне от 6 до 70

In [ ]:
np.random.seed(242)
ages = np.random.randint(6, 70, 50)
ages

добавляем имена для групп

In [ ]:
ranges = [6, 12, 18, 35, 50, 70]
labels = ['Youth', 'Young Adult', 'Adult', 'Middle Aged', 'Retired persons']
agebins = pd.cut(ages, ranges, labels=labels)
agebins.describe()

разбиваем (используя квантили) на 5 групп с одинаковым количеством элементов

In [ ]:
qbin = pd.qcut(dist, 5)

найдём статистику по полученным группам

In [ ]:
qbin.describe()

пример использования qcut:

In [ ]:
sp500_copy = sp500.copy()
sp500_copy['Price_Group'], bins = pd.qcut(sp500_copy.Price, 
                                          5,
                                          labels=['group_'+str(i) for i in range(1, 6)],
                                          retbins=True)
sp500_copy.Price_Group

In [ ]:
bins

In [ ]:
sp500_copy.Price_Group.value_counts()

#### кумулятивные суммы

вычисляем кумулятивную сумму

In [ ]:
pd.Series([1, 2, 3, 4]).cumsum()

вычисляем кумулятивное произведение

In [ ]:
pd.Series([1, 2, 3, 4]).cumprod()

#### ранжирование

для примера:

In [ ]:
s = pd.Series([160, 165, 165, 170, 175], index=list('abcde'))
s

ранжируем значения

In [ ]:
s.rank()

#### относительное изменение

In [ ]:
omh[['MSFT']].head()

вычисляем относительнон изменение для MSFT (текущее значение с предыдущим)

In [ ]:
omh[['MSFT']].pct_change().head()

In [ ]:
(48.46 - 48.62)/48.62

### Оконные функции

Объект Rolling:

In [ ]:
r = omh.MSFT.rolling(3)

возможные операции:

In [ ]:
r.

скользящее среднее:

In [ ]:
r.mean()

первое значение:

In [ ]:
omh.MSFT.loc['2014-12-01':'2014-12-03'].mean()

второе:

In [ ]:
omh.MSFT.loc['2014-12-02':'2014-12-04'].mean()